# 🎲 Jump Diffusion Playground

Welcome! Grab the sliders below and conjure up your own jumpy path. Press **Simulate!** whenever you're ready for another roller‑coaster ride.

In [ ]:
# Instalación de dependencias necesarias
!pip install -q git+https://github.com/jdospina/jump-diffusion-estimation.git ipywidgets matplotlib seaborn

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
sys.path.append(os.path.join('..','src'))
from jump_diffusion import JumpDiffusionSimulator, JumpDiffusionEstimator
import ipywidgets as widgets
from IPython.display import display, clear_output

plt.style.use('seaborn-v0_8')

## 🎛 Play with the knobs

Use these sliders to control the model parameters:

- **μ (mu)** – drift of the process.
- **σ (sigma)** – volatility of the continuous part.
- **jump_prob** – probability of a jump at each step.
- **jump_scale** – scale (standard deviation) of jump sizes.
- **jump_skew** – skewness of the jump size distribution.

Give them a wiggle!

In [2]:
mu = widgets.FloatSlider(value=0.1, min=-0.5, max=0.5, step=0.01, description='μ')
sigma = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.01, description='σ')
jump_prob = widgets.FloatSlider(value=0.05, min=0.0, max=0.5, step=0.01, description='jump_prob')
jump_scale = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01, description='jump_scale')
jump_skew = widgets.FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, description='jump_skew')
button = widgets.Button(description='Simulate!', button_style='success')
controls = widgets.VBox([mu, sigma, jump_prob, jump_scale, jump_skew, button])
output = widgets.Output()

## 🧪 Simulation & Estimation

Hit the green button to forge a fresh path. We'll even take a guess at the hidden parameters!

In [3]:

def run_simulation(_=None):
    with output:
        clear_output(wait=True)
        sim = JumpDiffusionSimulator(
            mu=mu.value,
            sigma=sigma.value,
            jump_prob=jump_prob.value,
            jump_scale=jump_scale.value,
            jump_skew=jump_skew.value,
        )
        times, path, _ = sim.simulate_path(T=1.0, n_steps=100, x0=0.0, seed=0)
        fig, ax = plt.subplots(figsize=(10,4))
        ax.plot(times, path, label='path')
        jump_idx = sim.last_jump_times
        if len(jump_idx) > 0:
            ax.scatter(times[jump_idx], path[jump_idx], color='red', label='jumps', zorder=5)
        ax.set_xlabel('time')
        ax.set_ylabel('value')
        ax.legend()
        plt.close(fig)
        display(fig)
        est = JumpDiffusionEstimator(np.diff(path), dt=times[1]-times[0])
        params = est.estimate(method='L-BFGS-B', options={'maxiter': 50})['parameters']
        print('Estimated parameters:')
        for k, v in params.items():
            print("{}: {:.3f}".format(k, v))

button.on_click(run_simulation)
display(widgets.HBox([controls, output]))
run_simulation()
